In [1]:
%%capture
!pip install numpy==1.21.4
!pip install pandas==1.3.4
!pip install loguru==0.5.3
!pip install psycopg2-binary

In [4]:
import sys
import os
from tqdm import tqdm
import pandas as pd
from adding_tables_psycopg import AddingDataPsycopg
import warnings
warnings.filterwarnings("ignore")

In [5]:
data_dir = './'
final_dir = './result'
os.makedirs(final_dir, exist_ok=True)
regions = [i for i in os.listdir(data_dir) if (len(i) == 2) and (not i in ['88', '81', '82', '80', '84', '85'])]

In [6]:
for r in regions:
    if (not f'{r}_hadobj.csv' in os.listdir(final_dir)) or (not f'{r}_parsed_chains.csv' in os.listdir(final_dir)):
        %run gar_try_v1.py {data_dir} {r} {final_dir}

./ ./09 ./result
chready: True


100%|██████████| 111828/111828 [00:00<00:00, 178560.04it/s]


In [3]:
%run gar_try_v1.py {data_dir} 23 {final_dir}

./ ./23 ./result
chready: True


100%|██████████| 1761860/1761860 [00:13<00:00, 135218.69it/s]


In [5]:
done = 0
for r in regions:
    if (f'{r}_hadobj.csv' in os.listdir(final_dir)) and (f'{r}_parsed_chains.csv' in os.listdir(final_dir)):
        done +=1
print(f'done: {done}')
print(f'done: {done/len(regions)*100}%')

done: 86
done: 100.0


In [15]:
sample = AddingDataPsycopg()
sample.create_table('hadobj', 'gar_source')
sample.create_table('parsed_chains', 'gar_source')

total_hadobj = 0
total_parsed_chains = 0

for r in tqdm(regions):
    if f'{r}_hadobj.csv' in os.listdir(final_dir):     
        hadobj = pd.read_csv(os.path.join(final_dir, f'{r}_hadobj.csv'), dtype={#'isactive': 'Int64', 'isactual': 'Int64',
                                                       'level': 'Int64', 'objectid': 'Int64',
                                                        'OKTMO' : 'str', 'KLADR' : 'str'})
        total_hadobj += len(hadobj)
        sample.write_to_sql(hadobj, 'hadobj', 'gar_source')
        #print(r)
        #print(f'{r}_hadobj({len(hadobj)}) - loaded')
    if f'{r}_parsed_chains.csv' in os.listdir(final_dir):
        parsed_chains = pd.read_csv(os.path.join(final_dir, f'{r}_parsed_chains.csv'), dtype={"l1": 'Int64', "l2": 'Int64', "l3": 'Int64',
                                                                     "l4": 'Int64', "l5": 'Int64', "l6": 'Int64',
                                                                     "l7": 'Int64', "l8": 'Int64', "l9": 'Int64',
                                                                     "l10": 'Int64',
                                                                     'tow': 'Int64',
                                                                     'mun': 'Int64',
                                                                     'id_tow': 'Int64',
                                                                     'id_mun': 'Int64',
                                                                     'id_reg': 'Int64',
                                                                     'street': 'Int64'})#.rename(columns={'left':'leftover'})   
        total_parsed_chains += len(parsed_chains)
        sample.write_to_sql(parsed_chains, 'parsed_chains', 'gar_source')
        #print(f'{r}_parsed_chains({len(parsed_chains)}) - loaded')
    #break
print(f'total_hadobj: {total_hadobj}')
print(f'total_parsed_chains: {total_parsed_chains}')

  8%|▊         | 7/86 [01:04<11:16,  8.56s/it]/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13,19,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
  9%|▉         | 8/86 [01:11<10:29,  8.08s/it]/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,13,19,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 13%|█▎        | 11/86 [01:37<11:08,  8.92s/it]/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1,12,13,18,19,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 19%|█▊        | 16/86 [02:48<16:09, 13.85s/it]/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: D

total_hadobj: 32453568
total_parsed_chains: 32448994
